# Solar Atmosphere Building: Basics

This script contains the basic ingredients for building solar atmospheres and uses

https://github.com/SWAT-Sheffield/pysac

http://pysac.readthedocs.io/en/latest/reference/index.html

Makes use of astropy tables, astropy constants and spline interpolation with scipy.

We use the McWhirter and VALIIIc datasets to construct solar atmospheres in hydrostatic and magnetohydrostatic  (eventually) equilibrium


In [1]:
import os
import numpy as np

import astropy.table
from astropy.table import Table
import astropy.units as u
from astropy.constants import k_B, m_p
import scipy.constants as asc
from astropy.table import Table


from scipy.interpolate import UnivariateSpline

#file to save configuration
filename='../../../configs/sac_test_asc.ini'

#called as
#alldat,modelinfo=read_sac_ascii('../../../configs/sac_test_asc.ini')


Set model dimensions and size of 3D grid

In [3]:
#if 1D or 2D set unused dimensions to 0, and unrequired xyz limits to 1. 
Nxyz = [129,129,129] # 3D grid
xyz_SI = [-1e6,1e6,-1e6,1e6,3.5e4,1.6e6] # xyz limits SI/CGS units 
xyz  = [-1.27*u.Mm,1.27*u.Mm,-1.27*u.Mm,1.27*u.Mm,0.*u.Mm,8.62*u.Mm] #grid size
#xyz.unit=u.Mm
#xyz=np.array([-1e6,1e6,-1e6,1e6,3.5e4,1.6e6])
#xyz = [-1*u.Mm,1e6,-1*u.Mm,1*u.Mm,0.035*u.Mm,1.6*u.Mm] # xyz limits SI/CGS units  
#xyz=xyz_SI

Note astropy quantity objects are used below the value method is used to return the current unit and value can be accessed via the unit and value attributes

http://docs.astropy.org/en/stable/units/quantity.html

In [4]:
"""
get_coords returns a non-dimensional dictionary describing the domain
coordinates.
"""
dz=(xyz[5]-xyz[4])/(Nxyz[2]-1)
#Z    = u.Quantity(np.linspace(xyz[4].value, xyz[5].value, Nxyz[2]), unit=xyz.unit)
Z    = u.Quantity(np.linspace(xyz[4].value, xyz[5].value, Nxyz[2]))
#Zext = u.Quantity(np.linspace(Z.min().value-4.*dz.value, Z.max().value+4.*dz.value, Nxyz[2]+8), unit=Z.unit)
Zext = u.Quantity(np.linspace(Z.min().value-4.*dz.value, Z.max().value+4.*dz.value, Nxyz[2]+8))
coords = {
              'dx':(xyz[1]-xyz[0])/(Nxyz[0]-1),
              'dy':(xyz[3]-xyz[2])/(Nxyz[1]-1),
              'dz':(xyz[5]-xyz[4])/(Nxyz[2]-1),
              'xmin':xyz[0],
              'xmax':xyz[1],
              'ymin':xyz[2],
              'ymax':xyz[3],
              'zmin':xyz[4],
              'zmax':xyz[5],
              'Z':Z,
              'Zext':Zext
            }

In [6]:
#identify location of source data files
__files__=''
#homedir = os.environ['HOME']
cwd = os.path.dirname(__files__)
#cwd = homedir+'/Dropbox/multi_tube/python/allpapers/'
VAL_file = os.path.join(cwd, 'VALIIIC.dat')
MTW_file = os.path.join(cwd, 'mcwhirter.dat')

filenames = [VAL_file, MTW_file]
# uncomment and switch to l_const/l_sqrt/l_linear/l_square as required  
#logical_pars['l_square'] = True 

In [7]:
#============================================================================
    # Dimensional units in terms of SI
#============================================================================
scales   = {
            'length':         1e2*u.Mm,
            'density':        1e-4*u.kg/u.m**3,
            'velocity':       1e2*u.m/u.s,
            'temperature':    1.0*u.K, 
            'magnetic':       1e-3*u.T #mT
           }
scales['energy density'] = scales['density'] * scales['velocity']**2
scales['time'] = scales['length'] / scales['velocity'] 
scales['mass'] = scales['density'] * scales['length']**3 
scales['force density'] = scales['density'] * scales['velocity'] / \
scales['time'] #D momentum/Dt force density balance 

In [8]:
"""
    Read in the data from Table 12 in Vernazza (1981) and combine with
    McWhirter (1975).

    Parameters
    ----------
    VAL_file : string
        The data file for the VAL3c atmosphere, defaults to
        `pysac.mhs_atmosphere.hs_model.VALIIIc_data`

    MTW_file : string
        The data file for the McWhirter atmosphere, defaults to
        `pysac.mhs_atmosphere.hs_model.MTWcorona_data`, if ``False`` is specified
        only the VAL atmosphere is returned.

    mu : float
        The mean molecular weight ratio for the corona. defaults to 0.6.

    Returns
    -------
    data : `astropy.table.Table`
        The combined data, sorted by Z.
"""
mu=0.602
VAL_file='VALIIIC.dat'
VAL3c = Table.read(VAL_file, format='ascii', comment='#', names=['Z', 'rho','p','T','n_i','n_e','mu'])

#print VAL3c
VAL3c['Z'].unit = u.km
VAL3c['rho'].unit = u.Unit('g cm-3')
VAL3c['p'].unit = u.Unit('dyne/cm^2')
VAL3c['T'].unit = u.K
VAL3c['n_i'].unit = u.one/u.cm**3
VAL3c['n_e'].unit = u.one/u.cm**3
# Calculate the mean molecular weight ratio
VAL3c['mu'] = 4.0/(3*0.74+1+VAL3c['n_e']/VAL3c['n_i'])
#    VAL3c['mu'] = 4.0/(3*0.74+1+VAL3c['n_e'].quantity/VAL3c['n_i'].quantity)
#print VAL3c['mu']
MTW_file='mcwhirter.dat'  
MTW = Table.read(MTW_file, format='ascii', comment='#', names=['Z', 'T','p','rho'])
#MTW = Table.read(MTW_file, format='ascii', comment='#')
#print MTW
MTW['Z'].unit = u.km
MTW['T'].unit = u.K
MTW['p'].unit = u.Unit('dyne cm-2')
MTW['rho'] = (MTW['p'] / k_B / MTW['T'] * m_p * mu).to('g cm-3')

#MTW['mu'] = mu

data = astropy.table.vstack([VAL3c, MTW], join_type='inner')
data.sort('Z')
#    data = astropy.table.vstack([VAL3c, MTW], join_type='inner')
print data   
    

   Z              rho               p         T    
   km           g / cm3         dyn / cm2     K    
-------- ---------------------- --------- ---------
   -75.0              3.192e-07  179000.0    8320.0
   -50.0               3.08e-07  157500.0    7610.0
   -25.0              2.949e-07  136800.0    6910.0
     0.0              2.727e-07  117200.0    6420.0
    50.0              2.152e-07   82740.0    5840.0
   100.0              1.606e-07   58040.0    5455.0
   150.0               1.15e-07   39260.0    5180.0
   250.0              5.413e-08   16910.0    4780.0
   350.0              2.334e-08    6798.0    4465.0
   450.0              9.327e-09    2569.0    4220.0
     ...                    ...       ...       ...
  3400.0  1.938251706627349e-15     0.118  444000.0
  3900.0  1.657518793803049e-15     0.115  506000.0
  5860.0 1.2061203216730686e-15     0.107  647000.0
  8790.0  9.132529404842348e-16    0.0988  789000.0
 13200.0  6.935044481564004e-16    0.0891  937000.0
 19800.0  5.

In [9]:
print (xyz[1]-xyz[0])/(Nxyz[0]-1)

0.01984375 Mm


In [10]:
from scipy.interpolate import UnivariateSpline
s=0.25
hdata = np.array(u.Quantity(data['Z']).to(u.m))
# interpolate total pressure, temperature and density profiles
pdata_f = UnivariateSpline(hdata,np.array(np.log(data['p'])),k=1, s=s)
Tdata_f = UnivariateSpline(hdata,np.array(np.log(data['T'])),k=1, s=s)
rdata_f = UnivariateSpline(hdata,np.array(np.log(data['rho'])),k=1, s=s)
#s=0.0 to ensure all points are strictly used for ionisation state
#muofT_f = UnivariateSpline(hdata,np.array(np.log(data['mu'])),k=1, s=0.0)
#print pdata_f(Z)
#print Z
print scales['temperature']
print(Tdata_f(hdata))
#print((data['Z']))
print Nxyz[0]
outdata = Table([np.zeros((Nxyz[0])),np.zeros((Nxyz[0])),np.zeros((Nxyz[0])),np.zeros((Nxyz[0]))],names=['Z', 'T','p','rho'])
outdata['Z'] = Z
#outdata['p'] = np.exp(pdata_f(Z.to(u.m))) 
#outdata['p'] = np.exp(pdata_f(Z.to(u.cm))) * data['p'].unit
outdata['p'] = np.exp(pdata_f(Z*1.0e8)) * data['p'].unit
#outdata['T'] = np.exp(Tdata_f(Z.to(u.m))) * data['T'].unit
#outdata['T'] = np.exp(Tdata_f()) * data['T'].unit 
outdata['T'] = np.exp(Tdata_f(Z*1.0e6)) * data['T'].unit
#outdata['rho'] = np.exp(rdata_f(Z.to(u.m))) * data['rho'].unit
#outdata['rho'] = np.exp(rdata_f()) * data['rho'].unit 
outdata['rho'] = np.exp(rdata_f(Z*1.0e8)) * data['rho'].unit 
#outdata['mu'] = np.exp(muofT_f(Z.to(u.m))) * u.one
print outdata


1.0 K
[ 8.93174722  8.8930591   8.85437097  8.81568285  8.73830661  8.66093036
  8.58355412  8.42880163  8.27404914  8.3422776   8.38662609  8.41391748
  8.44803171  8.48214594  8.51626017  8.5503744   8.61860286  8.63169179
  8.65132519  8.67357637  8.70368091  8.72985877  8.75603663  8.79137675
  8.81493682  8.86205697  8.89870598  8.91572159  8.92252783  8.93142831
  8.93666388  8.93928167  8.94189945  9.31143004  9.39061517  9.53115427
  9.81223248  9.82559102  9.85898735  9.91910075 10.12615803 10.11589002
 10.10818901 10.1017715  10.09971789 10.26595525 10.4321926  10.55687061
 10.80622664 11.3645516  11.81121156 12.63396845 12.72829267 12.88524633
 12.96372316 13.04219999 13.23839206 13.33382878 13.47649692 13.69122953
 14.01259806 14.06745975 14.1494752  14.27305255 14.45703316]
129
         Z                  T          ...          rho          
                            K          ...        g / cm3        
------------------- ------------------ ... ----------------------


In [80]:
#VAL3c = np.loadtxt(filenames[0])[::-1,:]
#VAL3c[:,0] *= 1e3 #km -> m
#VAL3c[:,1] *= 1e3 #g/cm^3 -> kg/m^3
#VAL3c[:,2] /= 1e1 #dyne/cm^2 -> Pascals(N/m^2)
#muTV=4.0/(3*0.74+1+VAL3c[:,6]/VAL3c[:,5])
#nzv=VAL3c[:,0].size
#MTW =np.loadtxt(filenames[1])[::-1,:]
#MTW[:,0] *= 1e3 #km -> m
#MTW[:,1] *= 1   #Kelvin
#MTW[:,2] /= 1e1 #dyne/cm^2 -> Pascals(N/m^2)
#nzm=MTW[:,0].size
#Combine both sets into single array
#hdata=np.zeros(nzv+nzm)
#hdata[0:nzv]=VAL3c[:,0]
#hdata[nzv:nzv+nzm]=MTW[:,0]
#hdata /= scales['length']
    
#pdata=np.zeros(nzv+nzm)
#pdata[0:nzv]=VAL3c[:,2]
#pdata[nzv:nzv+nzm]=MTW[:,2]
#pdata /= scales['energy density']
    
#Tdata=np.zeros(nzv+nzm)
#Tdata[0:nzv]=VAL3c[:,3]
#Tdata[nzv:nzv+nzm]=MTW[:,1] 
#Tdata /= scales['temperature']
    
#rdata=np.zeros(nzv+nzm)
#rdata[0:nzv]=VAL3c[:,1]
#rdata /= scales['density']   
#MTW[:,2], kB, mp and mu are in code units so no rescale needed
#rdata[nzv:nzv+nzm] = MTW[:,2]/physicalconstants['boltzmann']/MTW[:,1] *physicalconstants['proton_mass']*physicalconstants['mu']
    
#muofT = np.zeros(nzv+nzm) # mean molecular weight
#muofT[0:nzv] = muTV
#muofT[nzv:nzv+nzm] = physicalconstants['mu']
# interpolate total pressure, temperature and density profiles
#pdata_f = UnivariateSpline(hdata,np.log(pdata),k=1, s=0.25)
#Tdata_f = UnivariateSpline(hdata,np.log(Tdata),k=1, s=0.25)
#rdata_f = UnivariateSpline(hdata,np.log(rdata),k=1, s=0.25)
#s=0.0 to ensure all points are strictly used for ionisation state
#muofT_f = UnivariateSpline(hdata,np.log(muofT),k=1, s=0.0)
#pdata_i = np.exp(pdata_f(Z))
#Tdata_i = np.exp(Tdata_f(Z))
#rdata_i = np.exp(rdata_f(Z))
#muofT_i = np.exp(muofT_f(Z))

#source_data = [VAL3c,MTW] 

In [11]:
print Tdata_f(Z)

[8.81568285 8.81568275 8.81568264 8.81568254 8.81568243 8.81568233
 8.81568223 8.81568212 8.81568202 8.81568191 8.81568181 8.8156817
 8.8156816  8.8156815  8.81568139 8.81568129 8.81568118 8.81568108
 8.81568097 8.81568087 8.81568077 8.81568066 8.81568056 8.81568045
 8.81568035 8.81568024 8.81568014 8.81568004 8.81567993 8.81567983
 8.81567972 8.81567962 8.81567952 8.81567941 8.81567931 8.8156792
 8.8156791  8.81567899 8.81567889 8.81567879 8.81567868 8.81567858
 8.81567847 8.81567837 8.81567826 8.81567816 8.81567806 8.81567795
 8.81567785 8.81567774 8.81567764 8.81567754 8.81567743 8.81567733
 8.81567722 8.81567712 8.81567701 8.81567691 8.81567681 8.8156767
 8.8156766  8.81567649 8.81567639 8.81567628 8.81567618 8.81567608
 8.81567597 8.81567587 8.81567576 8.81567566 8.81567556 8.81567545
 8.81567535 8.81567524 8.81567514 8.81567503 8.81567493 8.81567483
 8.81567472 8.81567462 8.81567451 8.81567441 8.8156743  8.8156742
 8.8156741  8.81567399 8.81567389 8.81567378 8.81567368 8.81567358

In [ ]:




#open file to save configuration    
file = open(filename,'wb')

In [ ]:
header=''
nits=0
time=0
ndim=3
nvar=
nfields=
dim=[128 128 128]
head3=''
head4=''

modelinfo=(header,nits, time, ndim, nvar, nfields,dim,head3,head4)
#this script assumes data has been read using a routine such as sac-read3-ascii.py
#the following variables are assumed
#nits
#time
#ndim
#nvar
#nfields

#dim[2] or dim[3]

#gamma
#eta
#grav1
#grav2
#grav3

#all data is contained in an array alldat of shape nfields+ndim,dim[0],dim[1]


#write header lines

#header='sac_test_asc'
header=modelinfo[0]
#modelinfo=(header,nits, time, ndim, nvar, nfields,dim,head3,head4)
dim=[128,128]
ndim=2
nfields=12
time=modelinfo[2]
nits=modelinfo[1]
nvar=modelinfo[4]


head1=str(nits)+" "+str(time)+" "+str(ndim)+" "+str(nvar)+" "+str(nfields)

if ndim==2:
    head2=str(dim[0])+" "+str(dim[1])
elif ndim==3:
    head2=str(dim[0])+" "+str(dim[1])+" "+str(dim[2])    

#warning may need to explicityly write the adiabatic parameter and correct gravitational parameters here
head3="1.66667E+00  0.00000E+00  1.00000E+00  0.00000E+00  0.00000E+00  0.00000E+00"

if ndim==2:
    head4="x y h m1 m2 e b1 b2 eb rhob bg1 bg2   gamma eta   grav1 grav2"
elif ndim==3:
    head4="x y z h m1 m2 m3 e b1 b2 b3 eb rhob bg1 bg2 bg3   gamma eta   grav1 grav2 grav3"

In [ ]:
file.write(header)
file.write(head1+"\n")     
file.write(head2+"\n")
file.write(head3+"\n")
file.write(head4+"\n")     
    
if ndim==3:    
    for i3 in range(dim[2]):
        for i2 in range(dim[1]):
            for i1 in range(dim[0]):
                line=""
                for j in range(ndim+nfields):
                    line=line+str(alldat[i1,i2,i3,j])
                line=line+"\n"
                file.write(line)
                
if ndim==2:    
    for i2 in range(dim[1]):
        for i1 in range(dim[0]):
            line=""
            for j in range(ndim+nfields):              
                line=line+" "+str(newalldatslice[i1,i2,j])
            line=line+"\n"
            file.write(line)
            
file.close()            